In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from time import time

DATA_PATH = "../data/cifar-10-batches-py/"

def unpack(fname):
    import pickle
    with open(fname, "rb") as fin:
        dumped = pickle.load(fin, encoding="bytes")
    return dumped

train_input = []
train_output = []
for i in range(5):
    raw_train_data = unpack(DATA_PATH + "data_batch_" + str(i+1))
    train_input.append(np.array(raw_train_data[b"data"], dtype=float) / 255)
    train_output.append(np.array([[1 if x == label else 0 for x in range(10)] for label in raw_train_data[b"labels"]]))
train_input = np.concatenate(train_input).reshape(50000, 3, 32, 32).transpose(0, 2, 3, 1)
train_output = np.concatenate(train_output)

raw_test_data = unpack(DATA_PATH + "test_batch")
test_input = (np.array(raw_test_data[b"data"], dtype=float) / 255).reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1)
test_output = np.array(raw_test_data[b"labels"])

In [2]:
print(train_input.shape)
print(train_output.shape)

(50000, 32, 32, 3)
(50000, 10)


In [3]:
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko

Using TensorFlow backend.


In [4]:
model = km.Sequential()
model.add(kl.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(kl.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(kl.MaxPooling2D(pool_size=(2, 2)))
model.add(kl.Dropout(0.25))
model.add(kl.Flatten())
model.add(kl.Dense(128, activation='relu'))
model.add(kl.Dropout(0.5))
model.add(kl.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=ko.SGD(), metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [6]:
model.fit(train_input, train_output, epochs=5)

Epoch 1/5
50000/50000 [==============================] - 239s 5ms/step - loss: 1.8197 - acc: 0.3522
Epoch 2/5
50000/50000 [==============================] - 237s 5ms/step - loss: 1.5634 - acc: 0.4399
Epoch 3/5
50000/50000 [==============================] - 229s 5ms/step - loss: 1.4314 - acc: 0.4884
Epoch 4/5
50000/50000 [==============================] - 228s 5ms/step - loss: 1.3467 - acc: 0.5203
Epoch 5/5
50000/50000 [==============================] - 215s 4ms/step - loss: 1.2778 - acc: 0.5466


In [7]:
predict = [np.argmax(x) for x in model.predict(test_input)]

In [9]:
correct_sum = 0
for i in range(len(predict)):
    if predict[i] == test_output[i]:
        correct_sum += 1
print(correct_sum / 10000)

0.5916
